In [2]:
!pip install easyocr
!apt-get update
!apt-get install -y libstdc++6


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [3]:
!pip install flask flask-ngrok
!pip install --upgrade pyngrok flask-ngrok
!pip install -U bitsandbytes
!pip install flask-cors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.0 MB/s eta 0:00:00


In [ ]:
import easyocr
reader = easyocr.Reader(['en'])

In [ ]:
text = reader.readtext("/content/WhatsApp Image 2025-02-24 at 17.38.10.jpeg", detail=0)
text_str = ""
for i in text:
  text_str+=i
  text_str+=' '

In [ ]:
print(text_str)


Paracetamol 5oomg Citazin 250 ~4 


' Paracetamol, Citazin'

In [4]:
import re
import requests

def extract_medicine_info(text):
    # Regular expression to capture medicine names and their mg amounts
    pattern = r"\b([A-Za-z0-9\s-]+?)\s*-\s*(\d+\s*mg)\b"
    matches = re.findall(pattern, text)

    # Format extracted data
    medicines = [{"name": match[0].strip(), "dose": match[1].replace(" ", "")} for match in matches]

    return medicines

def call_together_ai(api_key, text):
    url = "https://api.together.xyz/v1/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "deepseek-ai/DeepSeek-V3",  # Replace with the actual model name
        "prompt": f"You have to first extract the possible medecine names from the text may have misspelled name you have to correct them and make sure that most of the part is matches with the given name give it in following format strictly without any extra names if there two possibility for one than give the most probable one gven names must be medicines:\n\n**Medicine Name:** [Complete Medicine Name] - [Dosage] mg\n\n{text}",
        "max_tokens": 300
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        try:
            result = response.json()
            refined_text = result.get("choices", [{}])[0].get("text", "").strip()
            return refined_text
        except (KeyError, IndexError, ValueError) as e:
            return f"Error parsing response: {e}"
    else:
        return f"Error: {response.status_code}, {response.text}"

# Example usage
api_key = "cd340778f43cae51b6894e5f153ab26047319c2dd1bfd13d5c2dbc3dcaa37bd2"  # Replace with your Together AI API key

# Step 1: Refine OCR Text
print(text_str)
refined_text = call_together_ai(api_key, text_str)
print("Refined Text:", refined_text)

# Step 2: Extract Medicine Names and Dosages
medicines = extract_medicine_info(refined_text)
print("Extracted Medicines:")
for med in medicines:
    print(med["name"], "-", med["dose"])


NameError: name 'text_str' is not defined

In [ ]:
import re
import requests
import easyocr
from flask import Flask, request, jsonify
from pyngrok import ngrok
ngrok.set_auth_token("2rl2aFvk15iMWC4MOqpiQUmopuF_3r8H1PQha6QJkTYDd3sNN")
app = Flask(__name__)
reader = easyocr.Reader(['en'])
from flask_cors import CORS
# Initialize CORS with default settings (allows all origins)
CORS(app)


API_KEY = "cd340778f43cae51b6894e5f153ab26047319c2dd1bfd13d5c2dbc3dcaa37bd2"

def extract_medicine_info(text):
    pattern = r"\b([A-Za-z0-9\s-]+?)\s*-\s*(\d+\s*mg)\b"
    matches = re.findall(pattern, text)
    medicines = [{"name": match[0].strip(), "dose": match[1].replace(" ", "")} for match in matches]
    return medicines

def call_together_ai(text):
    url = "https://api.together.xyz/v1/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "deepseek-ai/DeepSeek-R1",
        "prompt": f"You are a medicine name extractor and corrector. Your task is to analyze the provided text for any potential medicine names. Please follow these guidelines:1. **Extraction**: Identify any names that resemble medicine names within the text.2. **Correction**: If you come across names that appear to be misspelled, correct them to closely match the actual medicine name, ensuring they conform to known pharmaceutical standards.3. **Validation**: Ensure that only valid medicine names are provided, along with their corresponding correct dosages.4. **Selection**: In cases where multiple possible names for a medicine exist, select the one that seems most probable and accurate.5. **Output Format**: :\n\n**Medicine Name:** [Complete Medicine Name] - [Dosage] mg\n\n, with no additional information included no explaination:- Medicine Name 1: Dosage- Medicine Name 2: Dosage- (Continue for any additional medicine names identified.)Begin by processing the following input text:{text}",
        "max_tokens": 300
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        try:
            result = response.json()
            refined_text = result.get("choices", [{}])[0].get("text", "").strip()
            return refined_text
        except (KeyError, IndexError, ValueError) as e:
            return f"Error parsing response: {e}"
    else:
        return f"Error: {response.status_code}, {response.text}"

@app.route('/', methods=['GET'])
def home():
    return jsonify({"msg": "API is running"}), 200

@app.route('/extract_medicines', methods=['POST'])
def extract_medicines():
    if "image" not in request.files:
        return jsonify({"error": "No image file provided"}), 400

    image_file = request.files["image"]
    image_path = "/tmp/uploaded_image.png"
    image_file.save(image_path)

    text = reader.readtext(image_path, detail=0)
    text_str = " ".join(text)
    print("Extracted Text:", text_str)

    if not text_str:
        return jsonify({"error": "No text extracted"}), 400

    refined_text = call_together_ai(text_str)
    medicines = extract_medicine_info(refined_text)

    return jsonify({"refined_text": refined_text, "medicines": medicines})

if __name__ == '__main__':
    public_url = ngrok.connect(5000).public_url
    print(f"Ngrok tunnel: {public_url}")
    app.run(port=5000)


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteNgrok tunnel: https://ef5f-34-147-80-53.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Extracted Text: PARACETAMOL 560 wI NAP RDXEN 2 Sowvg ERythrohycin Loovv


INFO:werkzeug:127.0.0.1 - - [25/Feb/2025 18:54:10] "POST /extract_medicines HTTP/1.1" 200 -


...

INFO:werkzeug:127.0.0.1 - - [25/Feb/2025 20:36:30] "POST /extract_medicines HTTP/1.1" 200 -


Extracted Text: PARACETAMOL 560 wI NAP RDXEN 2 Sowvg ERythrohycin Loovv


INFO:werkzeug:127.0.0.1 - - [25/Feb/2025 20:45:27] "POST /extract_medicines HTTP/1.1" 200 -


Extracted Text: PARACETAMOL 560 wI NAP RDXEN 2 Sowvg ERythrohycin Loovv


INFO:werkzeug:127.0.0.1 - - [25/Feb/2025 20:47:26] "POST /extract_medicines HTTP/1.1" 200 -
